In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import h5py

def load_dataset(shape_img=(64, 64), show_samples=False):
    with h5py.File('dogs.h5', 'r') as hf:
        Xtrain = hf['Xtrain'][:]
        Xtest = hf['Xtest'][:]
        Ytrain = hf['Ytrain'][:]
        Ytest = hf['Ytest'][:]
    
    if show_samples:
        # Plot first 5 images of each dataset for verification
        fig, axes = plt.subplots(2, 10, figsize=(20, 4))
        for i in range(10):
            x = Xtrain[:, i].reshape(shape_img[0], shape_img[1], 3)
            axes[0, i].imshow(x)
            axes[0, i].axis('off')  # Hide the axis
            axes[0, i].set_title("Dog" if Ytrain[0, i] == 1 else "Car", fontsize=8)
            x = Xtest[:, i].reshape(shape_img[0], shape_img[1], 3)
            axes[1, i].imshow(x)
            axes[1, i].axis('off')  # Hide the axis
            axes[1, i].set_title("Dog" if Ytest[0, i] == 1 else "Car", fontsize=8)

        plt.show()

    # Torch needs X and Y to be of shape (m, num_features) and (m, 1) correspondingly!
    Xtrain = torch.tensor(Xtrain.T, dtype=torch.float32)
    Xtest = torch.tensor(Xtest.T, dtype=torch.float32)
    Ytrain = torch.tensor(Ytrain.T, dtype=torch.float32)
    Ytest = torch.tensor(Ytest.T, dtype=torch.float32)
    
    return Xtrain, Xtest, Ytrain, Ytest

class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LEARNING_RATE = 0.005
NUM_ITER = 100000
LABELS = ['Car', 'Dog']

# Load Train/Test datasets
Xtrain, Xtest, Ytrain, Ytest = load_dataset(show_samples=True)
x_n = Xtrain.shape[1]
Xtrain = Xtrain.to(device)
Ytrain = Ytrain.to(device)

# Define model, loss, and optimizer
model = LogisticRegressionModel(x_n).to(device)
criterion = nn.BCELoss()  # Binary cross-entropy loss
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE)

# Training loop
for epoch in range(NUM_ITER):
    optimizer.zero_grad()
    outputs = model(Xtrain)  # Forward pass
    loss = criterion(outputs, Ytrain)  # Loss calculation
    loss.backward()  # Backward pass
    optimizer.step()  # Update weights

    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')